In [ ]:
from plyfile import PlyData
import open3d as o3d
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import pandas as pd
from itertools import cycle

scene_ids = [
    "scene0000_00",
    "scene0000_01",
    "scene0000_02"
]

scan_root = "data/scans"

all_labels = set()
for scene_id in scene_ids:
    labels_path = f"{scan_root}/{scene_id}/{scene_id}_vh_clean_2.labels.ply"
    plydata = PlyData.read(open(labels_path, "rb"))
    label_field = next(
        (f for f in ['label', 'scalar_Label'] if f in plydata['vertex'].data.dtype.names),
        None
    )
    labels = np.array(plydata['vertex'][label_field])
    all_labels.update(labels.astype(str))

palette = px.colors.qualitative.Plotly  
color_cycle = cycle(palette)
color_map = {label: color for label, color in zip(sorted(all_labels), color_cycle)}

cols = 1
rows = len(scene_ids)
fig = make_subplots(
    rows=rows, cols=cols,
    specs=[[{'type': 'scene'}]] * rows,
    subplot_titles=scene_ids
)

for i, scene_id in enumerate(scene_ids):
    mesh_path = f"{scan_root}/{scene_id}/{scene_id}_vh_clean_2.ply"
    labels_path = f"{scan_root}/{scene_id}/{scene_id}_vh_clean_2.labels.ply"

    mesh = o3d.io.read_point_cloud(mesh_path)
    points = np.asarray(mesh.points)

    plydata = PlyData.read(open(labels_path, "rb"))
    label_field = next(
        (f for f in ['label', 'scalar_Label'] if f in plydata['vertex'].data.dtype.names),
        None
    )
    labels = np.array(plydata['vertex'][label_field])

    df = pd.DataFrame({
        "x": points[:, 0],
        "y": points[:, 1],
        "z": points[:, 2],
        "label": labels.astype(str),
    })

    temp_fig = px.scatter_3d(
        df, x="x", y="y", z="z",
        color="label",
        color_discrete_map=color_map,
        opacity=0.7
    )

    for trace in temp_fig.data:
        fig.add_trace(trace, row=i+1, col=1)

fig.update_layout(
    height=900 * cols,
    width=800,
    title_text="3D Point Clouds with Ground Truth Labels (Consistent Coloring)",
    margin=dict(l=10, r=10, b=10, t=40),
    showlegend=False
)

for trace in fig.data:
    trace.marker.size = 1

fig.show()

In [ ]:
from plyfile import PlyData
import open3d as o3d
import numpy as np
import pandas as pd
import plotly.express as px
import torch
from plotly.subplots import make_subplots
from itertools import cycle

scene_ids = [
    "scene0000_00",
]

scan_root = "data/scans"
prediction_root = "outputs/sam2_hiera_s_test_no_pngs"

all_pred_ids = set()
for scene_id in scene_ids:
    pred_path = f"{prediction_root}/{scene_id}.pth"
    preds = torch.load(pred_path)
    instance_ids = preds.numpy() if isinstance(preds, torch.Tensor) else preds
    all_pred_ids.update(np.unique(instance_ids).astype(str))

palette = px.colors.qualitative.Plotly
color_cycle = cycle(palette)
color_map = {pid: color for pid, color in zip(sorted(all_pred_ids), color_cycle)}

cols = 2
rows = len(scene_ids)
fig = make_subplots(
    rows=rows, cols=cols,
    specs=[[{'type': 'scene'}, {'type': 'scene'}]] * rows,
    subplot_titles=[f"{scene_ids[i//2]} - GT" if i % 2 == 0 else f"{scene_ids[i//2]} - Pred" for i in range(2 * len(scene_ids))]
)

for i, scene_id in enumerate(scene_ids):
    mesh_path = f"{scan_root}/{scene_id}/{scene_id}_vh_clean_2.ply"
    labels_path = f"{scan_root}/{scene_id}/{scene_id}_vh_clean_2.labels.ply"
    prediction_path = f"{prediction_root}/{scene_id}.pth"

    mesh = o3d.io.read_point_cloud(mesh_path)
    points = np.asarray(mesh.points)

    plydata = PlyData.read(open(labels_path, "rb"))
    label_field = next((f for f in ['label', 'scalar_Label'] if f in plydata['vertex'].data.dtype.names), None)
    labels = np.array(plydata['vertex'][label_field])

    instance_ids = torch.load(prediction_path)
    instance_ids = instance_ids.numpy() if isinstance(instance_ids, torch.Tensor) else instance_ids

    df_gt = pd.DataFrame({
        "x": points[:, 0],
        "y": points[:, 1],
        "z": points[:, 2],
        "label": labels.astype(str),
    })
    fig_gt = px.scatter_3d(df_gt, x="x", y="y", z="z", color="label", opacity=0.7)
    for trace in fig_gt.data:
        fig.add_trace(trace, row=i+1, col=1)

    df_pred = pd.DataFrame({
        "x": points[:, 0],
        "y": points[:, 1],
        "z": points[:, 2],
        "id": instance_ids.astype(str)
    })
    fig_pred = px.scatter_3d(df_pred, x="x", y="y", z="z", color="id", color_discrete_map=color_map, opacity=0.7)
    for trace in fig_pred.data:
        fig.add_trace(trace, row=i+1, col=2)

fig.update_layout(
    height=500 * rows,
    width=800,
    title_text="3D Point Clouds: GT Labels vs Predicted Instance IDs",
    margin=dict(l=10, r=10, b=10, t=40),
    showlegend=False
)

for trace in fig.data:
    trace.marker.size = 1

fig.show()

In [ ]:
from plyfile import PlyData
import open3d as o3d
import numpy as np
import pandas as pd
import plotly.express as px
import torch
from plotly.subplots import make_subplots
from itertools import cycle

def oracle_classes(labels, results):
    ids = np.unique(results)
    oracle_labels = {}
    for id in ids:
        index = np.where(results == id)[0]
        unique, counts = np.unique(labels[index], return_counts=True)
        oracle_label = unique[np.argmax(counts)]
        oracle_labels[id] = oracle_label
    return oracle_labels

scene_ids = [
    "scene0000_00",
]

scan_root = "data/scans"
prediction_root = "outputs/sam2_hiera_s_test_no_pngs"

all_labels = set()
for scene_id in scene_ids:
    labels_path = f"{scan_root}/{scene_id}/{scene_id}_vh_clean_2.labels.ply"
    plydata = PlyData.read(open(labels_path, "rb"))
    label_field = next((f for f in ['label', 'scalar_Label'] if f in plydata['vertex'].data.dtype.names), None)
    labels = np.array(plydata['vertex'][label_field])
    all_labels.update(labels.astype(str))

palette = px.colors.qualitative.Plotly
color_cycle = cycle(palette)
color_map = {label: color for label, color in zip(sorted(all_labels), color_cycle)}

cols = 2
rows = len(scene_ids)
fig = make_subplots(
    rows=rows, cols=cols,
    specs=[[{'type': 'scene'}, {'type': 'scene'}]] * rows,
    subplot_titles=[f"{scene_ids[i//2]} - GT" if i % 2 == 0 else f"{scene_ids[i//2]} - Oracled" for i in range(2 * len(scene_ids))]
)

for i, scene_id in enumerate(scene_ids):
    mesh_path = f"{scan_root}/{scene_id}/{scene_id}_vh_clean_2.ply"
    labels_path = f"{scan_root}/{scene_id}/{scene_id}_vh_clean_2.labels.ply"
    prediction_path = f"{prediction_root}/{scene_id}.pth"

    mesh = o3d.io.read_point_cloud(mesh_path)
    points = np.asarray(mesh.points)

    plydata = PlyData.read(open(labels_path, "rb"))
    label_field = next((f for f in ['label', 'scalar_Label'] if f in plydata['vertex'].data.dtype.names), None)
    labels = np.array(plydata['vertex'][label_field])

    instance_ids = torch.load(prediction_path)
    instance_ids = instance_ids.numpy() if isinstance(instance_ids, torch.Tensor) else instance_ids

    id_to_label = oracle_classes(labels, instance_ids)
    oracled_labels = np.array([id_to_label[iid] for iid in instance_ids])

    df_gt = pd.DataFrame({
        "x": points[:, 0],
        "y": points[:, 1],
        "z": points[:, 2],
        "label": labels.astype(str),
    })
    fig_gt = px.scatter_3d(df_gt, x="x", y="y", z="z", color="label", color_discrete_map=color_map, opacity=0.7)
    for trace in fig_gt.data:
        fig.add_trace(trace, row=i+1, col=1)

    df_oracle = pd.DataFrame({
        "x": points[:, 0],
        "y": points[:, 1],
        "z": points[:, 2],
        "label": oracled_labels.astype(str)
    })
    fig_pred = px.scatter_3d(df_oracle, x="x", y="y", z="z", color="label", color_discrete_map=color_map, opacity=0.7)
    for trace in fig_pred.data:
        fig.add_trace(trace, row=i+1, col=2)

fig.update_layout(
    height=500 * rows,
    width=800,
    title_text="3D Point Clouds: Ground Truth vs Oracled Predictions (Same Color Map)",
    margin=dict(l=10, r=10, b=10, t=40),
    showlegend=False
)

for trace in fig.data:
    trace.marker.size = 1

fig.show()

In [ ]:
from plyfile import PlyData
import open3d as o3d
import numpy as np
import pandas as pd
import plotly.express as px
import torch
from plotly.subplots import make_subplots
from itertools import cycle

def oracle_classes(labels, results):
    ids = np.unique(results)
    oracle_labels = {}
    for id in ids:
        index = np.where(results == id)[0]
        unique, counts = np.unique(labels[index], return_counts=True)
        oracle_label = unique[np.argmax(counts)]
        oracle_labels[id] = oracle_label
    return oracle_labels

# Configurable inputs
scene_ids = ["scene0000_01"]
scan_root = "data/scans"
prediction_root_1 = "outputs/sam2_hiera_t_efficient_test"
prediction_root_2 = "outputs/sam2_hiera_t_test"

# Create shared color map
all_labels = set()
for scene_id in scene_ids:
    labels_path = f"{scan_root}/{scene_id}/{scene_id}_vh_clean_2.labels.ply"
    plydata = PlyData.read(open(labels_path, "rb"))
    label_field = next((f for f in ['label', 'scalar_Label'] if f in plydata['vertex'].data.dtype.names), None)
    labels = np.array(plydata['vertex'][label_field])
    all_labels.update(labels.astype(str))

palette = px.colors.qualitative.Plotly
color_cycle = cycle(palette)
color_map = {label: color for label, color in zip(sorted(all_labels), color_cycle)}

cols = 2
rows = len(scene_ids)
fig = make_subplots(
    rows=rows, cols=cols,
    specs=[[{'type': 'scene'}] * cols] * rows,
    subplot_titles=[
        f"{scene_ids[i]} - GT" if j == 0 else
        f"{scene_ids[i]} - Pred 1" if j == 1 else
        f"{scene_ids[i]} - Pred 2"
        for i in range(len(scene_ids)) for j in range(3)
    ]
)

for i, scene_id in enumerate(scene_ids):
    mesh_path = f"{scan_root}/{scene_id}/{scene_id}_vh_clean_2.ply"
    labels_path = f"{scan_root}/{scene_id}/{scene_id}_vh_clean_2.labels.ply"
    prediction_path_1 = f"{prediction_root_1}/{scene_id}.pth"
    prediction_path_2 = f"{prediction_root_2}/{scene_id}.pth"

    mesh = o3d.io.read_point_cloud(mesh_path)
    points = np.asarray(mesh.points)

    plydata = PlyData.read(open(labels_path, "rb"))
    label_field = next((f for f in ['label', 'scalar_Label'] if f in plydata['vertex'].data.dtype.names), None)
    labels = np.array(plydata['vertex'][label_field])

    def load_oracled_preds(pred_path):
        instance_ids = torch.load(pred_path)
        if isinstance(instance_ids, torch.Tensor):
            instance_ids = instance_ids.numpy()
        id_to_label = oracle_classes(labels, instance_ids)
        return np.array([id_to_label[iid] for iid in instance_ids])

    pred1_labels = load_oracled_preds(prediction_path_1)
    pred2_labels = load_oracled_preds(prediction_path_2)

    def add_subplot(col, data_labels, title_suffix):
        df = pd.DataFrame({
            "x": points[:, 0],
            "y": points[:, 1],
            "z": points[:, 2],
            "label": data_labels.astype(str)
        })
        fig_local = px.scatter_3d(df, x="x", y="y", z="z", color="label",
                                  color_discrete_map=color_map, opacity=0.7)
        for trace in fig_local.data:
            fig.add_trace(trace, row=i+1, col=col)

    add_subplot(1, pred1_labels, "Pred1")
    add_subplot(2, pred2_labels, "Pred2")

fig.update_layout(
    height=500 * rows,
    width=800,
    title_text="3D Point Clouds: Ground Truth vs Predictions from Two Models",
    margin=dict(l=10, r=10, b=10, t=40),
    showlegend=False
)

for trace in fig.data:
    trace.marker.size = 1

fig.show()